In [29]:
import os
import pandas as pd
from distrito_federal_setor import setores
import re

# Caminho para o arquivo Excel
caminho_arquivo = r"D:\Estágio - TRIM\arquivos sujos\Imoveis a Venda_DF_Nov22.xlsx"

# Extrair o nome do arquivo sem a extensão
nome_arquivo, extensao_arquivo = os.path.splitext(os.path.basename(caminho_arquivo))

# Carregando o arquivo Excel em um DataFrame
df = pd.read_excel(caminho_arquivo)

# Verificar e remover imóveis duplicados com base no conteúdo do link
df.drop_duplicates(subset=["Link"], inplace=True)

# Verificar se há imóveis com "Preço" igual a zero ou nulo
preco_zero_null = df["Preço"].isnull() | (df["Preço"] == 0)

# Remover imóveis com "Preço" igual a zero ou nulo
df = df[~preco_zero_null]

# Verificar se há imóveis com "Área" igual a zero ou nula
area_zero_null = df["Área"].isnull() | (df["Área"] == 0)

# Remover imóveis com "Área" igual a zero ou nula
df = df[~area_zero_null]

# Remover imóveis com "Preço" igual a "Sob Consulta"
df = df[~(df["Preço"].str.lower() == "sob consulta")]

# Remover caracteres não numéricos da coluna "Preço" e converter para numérico
df["Preço"] = pd.to_numeric(
    df["Preço"].apply(lambda x: "".join(filter(str.isdigit, str(x)))), errors="coerce"
)


# Função para extrair e contar o número máximo de quartos
def extrair_max_quartos(texto):
    if isinstance(texto, str):  # Verifica se o valor é uma string
        quartos_matches = re.findall(
            r"(\d+)\s*quarto|(\d+)\s*quartos", texto, flags=re.IGNORECASE
        )
        if quartos_matches:
            return max(int(quarto) for quarto in sum(quartos_matches, ()))
    return pd.NA  # Retorna NaN se não houver correspondências


# Extrair e contabilizar o número máximo de quartos
df["Quartos"] = df["new-details-ul"].apply(extrair_max_quartos)
df["Quartos"].fillna(df["new-details-ul1"].apply(extrair_max_quartos), inplace=True)
df["Quartos"].fillna(df["new-details-ul2"].apply(extrair_max_quartos), inplace=True)
df["Quartos"] = df["Quartos"].astype(float)

# Convertendo as colunas "Área" e "Preço" para numéricos
df["Área"] = pd.to_numeric(df["Área"], errors="coerce")

# Remover linhas onde "Área" é NaN após a conversão
df = df.dropna(subset=["Área"])

# Verificar se há valores vazios nas colunas "Preço" e "Área" e excluir as linhas correspondentes
df = df.dropna(subset=["Preço", "Área"])

# Criando uma nova coluna 'M2' que é o resultado da divisão da coluna 'Preço' pela coluna 'Área'
df["M2"] = df["Preço"] / df["Área"]

# Salvar o DataFrame modificado no caminho desejado
caminho_arquivo_modificado = os.path.join(
    r"D:\Estágio - TRIM\arquivos limpos", f"{nome_arquivo}_limpo{extensao_arquivo}"
)
df.to_excel(caminho_arquivo_modificado, index=False)

print("\nArquivo salvo com sucesso em:", caminho_arquivo_modificado)

# Exibindo as primeiras linhas do DataFrame com as novas colunas de amenidades
print(df.head())

ValueError: invalid literal for int() with base 10: ''